# 🥈 Silver Layer – Cleansed & Enriched Data

The Silver layer contains cleaned, standardized, and schema‑validated data derived from the raw Bronze layer.  
Its purpose is to transform raw ingestion data into reliable, analytics‑ready datasets.

### Key characteristics
- Data is **cleaned** (duplicates removed, invalid records filtered)
- Schema is **standardised** and enforced
- Missing or malformed fields are handled
- Joins and light enrichment may be applied
- Data quality checks are introduced

### Typical tasks in this layer
- Parse and normalize semi‑structured data (JSON, CSV, logs)
- Apply business rules and data quality validations
- Deduplicate records using primary keys or natural keys
- Convert raw timestamps, formats, and data types
- Produce clean Delta tables for downstream Gold consumption


In [0]:
from pyspark.sql.functions import (
    col, trim, to_date, to_timestamp, from_unixtime, when, year, month
)

# Load Bronze
bronze_df = spark.table("bronze.sales_orders")

# Convert epoch seconds → timestamp safely
silver_df = bronze_df.withColumn(
    "order_timestamp",
    when(
        col("order_datetime").isNotNull() & (col("order_datetime") > 0),
        to_timestamp(from_unixtime(col("order_datetime")))
    ).otherwise(None)
)

# Add analytics-friendly date fields
silver_df = (
    silver_df
    .withColumn("order_date", to_date(col("order_timestamp")))
    .withColumn("order_year", year(col("order_timestamp")))
    .withColumn("order_month", month(col("order_timestamp")))
)

# Clean string fields
silver_df = (
    silver_df
    .withColumn("customer_name", trim(col("customer_name")))
    .withColumn("ordered_products", trim(col("ordered_products")))
    .withColumn("promo_info", trim(col("promo_info")))
)

# Deduplicate orders (business key)
silver_df = silver_df.dropDuplicates(["order_number"])

# Write Silver table
silver_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.sales_orders")


Checking the output

In [0]:
%sql
SELECT
  MIN(order_date),
  MAX(order_date),
  COUNT(*) AS rows
FROM silver.sales_orders;


MIN(order_date),MAX(order_date),rows
2019-08-01,2019-11-14,4000


In [0]:
%sql
SELECT * FROM silver.sales_orders LIMIT 5;


clicked_items,customer_id,customer_name,number_of_line_items,order_datetime,order_number,ordered_products,promo_info,ingestion_timestamp,order_timestamp,order_date,order_year,order_month
"[[""AVpfvr461cnluZ0-qgHR"",""99""],[""AVpiBT8VLJeJML43hWwi"",""79""],[""AVpgQP5vLJeJML43LQbd"",""92""],[""AVqkEzW9U2_QcyX9O0Gr"",""51""],[""AVqkEzW9U2_QcyX9O0Gr"",""38""]]",7442821,"FITZMAURICE, PATRICK",3,1565696372,317568493,"[{""curr"":""USD"",""id"":""AVpgQP5vLJeJML43LQbd"",""name"":""Rony Ultra-Portable Bluetooth Speaker"",""price"":""49"",""promotion_info"":null,""qty"":""2"",""unit"":""pcs""},{""curr"":""USD"",""id"":""AVqkEzW9U2_QcyX9O0Gr"",""name"":""Reagate - Backup Plus 5TB External USB 3.0 Portable Hard Drive - silver"",""price"":""147"",""promotion_info"":null,""qty"":""8"",""unit"":""pcs""}]",[],2026-01-07T06:42:45.854Z,2019-08-13T11:39:32.000Z,2019-08-13,2019,8
"[[""AVpfMVD-ilAPnD_xW6bu"",""19""],[""AVphTO5W1cnluZ0-Aygg"",""84""]]",17412651,"DUNLEAVY-GERSTER, MONICA T",2,1566085474,317568652,"[{""curr"":""USD"",""id"":""AVpfMVD-ilAPnD_xW6bu"",""name"":""Rony - BC-TRX Battery Charger - Black"",""price"":""27"",""promotion_info"":{""promo_disc"":0.03,""promo_id"":""0"",""promo_item"":""AVpfMVD-ilAPnD_xW6bu"",""promo_qty"":""1""},""qty"":""1"",""unit"":""pcs""},{""curr"":""USD"",""id"":""AVphTO5W1cnluZ0-Aygg"",""name"":""Adventura SH 140 II Shoulder Bag (Black)"",""price"":""31"",""promotion_info"":null,""qty"":""2"",""unit"":""pcs""}]","[{""promo_disc"":0.03,""promo_id"":""0"",""promo_item"":""AVpfMVD-ilAPnD_xW6bu"",""promo_qty"":""1""}]",2026-01-07T06:42:45.854Z,2019-08-17T23:44:34.000Z,2019-08-17,2019,8
"[[""AVpf-2hGilAPnD_xlfDv"",""87""],[""AVpffrToLJeJML43-M2g"",""49""],[""AVpfMVD-ilAPnD_xW6bu"",""35""]]",4952777,flying point digital,1,1569514376,317570136,"[{""curr"":""USD"",""id"":""AVpfMVD-ilAPnD_xW6bu"",""name"":""Rony - BC-TRX Battery Charger - Black"",""price"":""27"",""promotion_info"":{""promo_disc"":0.03,""promo_id"":""0"",""promo_item"":""AVpfMVD-ilAPnD_xW6bu"",""promo_qty"":""10""},""qty"":""10"",""unit"":""pcs""}]","[{""promo_disc"":0.03,""promo_id"":""0"",""promo_item"":""AVpfMVD-ilAPnD_xW6bu"",""promo_qty"":""10""}]",2026-01-07T06:42:45.854Z,2019-09-26T16:12:56.000Z,2019-09-26,2019,9
"[[""AVphIbvi1cnluZ0--vFy"",""37""],[""AVpfMVD-ilAPnD_xW6bu"",""28""],[""AVpfhULY1cnluZ0-mN6h"",""44""],[""AVpftMuHLJeJML43COhu"",""79""],[""AVqVGaCCU2_QcyX9Ozcf"",""86""],[""AVpfMVD-ilAPnD_xW6bu"",""23""]]",19580290,genesis electronics recycling,2,1571272463,317570871,"[{""curr"":""USD"",""id"":""AVqVGaCCU2_QcyX9Ozcf"",""name"":""15.4 NakBook Pro with Touch Bar (Late 2016, Space Gray)"",""price"":""27"",""promotion_info"":null,""qty"":""3"",""unit"":""pcs""},{""curr"":""USD"",""id"":""AVpfMVD-ilAPnD_xW6bu"",""name"":""Rony - BC-TRX Battery Charger - Black"",""price"":""2036"",""promotion_info"":{""promo_disc"":0.03,""promo_id"":""0"",""promo_item"":""AVpfMVD-ilAPnD_xW6bu"",""promo_qty"":""3""},""qty"":""3"",""unit"":""pcs""}]","[{""promo_disc"":0.03,""promo_id"":""0"",""promo_item"":""AVpfMVD-ilAPnD_xW6bu"",""promo_qty"":""3""}]",2026-01-07T06:42:45.854Z,2019-10-17T00:34:23.000Z,2019-10-17,2019,10
"[[""AVpfdBS41cnluZ0-lBIj"",""51""],[""AVpio2wSilAPnD_xGGln"",""32""]]",30544967,"GARRETT, BRADLEY N",2,1573481849,317571868,"[{""curr"":""USD"",""id"":""AVpfdBS41cnluZ0-lBIj"",""name"":""Details About Mogitech G920 Xbox Driving Force Racing Wheel For Xbox One And Pc (941000121)"",""price"":""293"",""promotion_info"":null,""qty"":""2"",""unit"":""pcs""},{""curr"":""USD"",""id"":""AVpio2wSilAPnD_xGGln"",""name"":""CRX-322 CD Receiver"",""price"":""249"",""promotion_info"":null,""qty"":""1"",""unit"":""pcs""}]",[],2026-01-07T06:42:45.854Z,2019-11-11T14:17:29.000Z,2019-11-11,2019,11


In [0]:
%sql
SELECT COUNT(*) FROM silver.sales_orders;


COUNT(*)
4000
